In [1]:
import sys

print(sys.version)

3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 08:58:31) [Clang 14.0.6 ]


#  디스크립터란 (Descriptor)

- 클래스에서 속성(attribute)의 접근, 할당, 삭제 등을 제어하는 프로토콜입니다. 
- 디스크립터를 사용하면 속성에 대한 동작을 커스터마이즈하거나 제한할 수 있습니다.

###  클래스의 속성을 보호하기 위해 만드는 특별한 클래스이다.
- 디스크립터로 만드는 속성을 보호 속성으로 만든다.
- 속성에 대한 접근을 이름으로 조회하고 갱신할 수 있다.

### 기본 프로토콜 규약에서는 3가지가 있다.

> `__get__`, `__set__`, `__delete__` 중 하나가 구현되어 있는 클래스를 디스크립터 클래스

> `__set_name__` 변수의 이름을 읽어서 처리하는 속성 

# 1.  디스크립터를 만들고 내부 인스턴스에 속성 넣기

## 1-1 디스크립터 클래스 정의

- 디스크립터 객체를 생성할 때 값을 관리하는 필드와 이 속성을 처리하는 3개의 메소드를 정의한다.
- 값 저장은 일단 디스크립터 객체에 만들어서 보관하는 것부터 알아본다.


### 디스크립트 클래스 정의

- get, set, delete 메서드 정의
- init에 디스크립트 객체에 값을 생성 

In [2]:
class Descriptor:
    def __init__(self, value):
        self.value = value                # 디스크립터 객체의 속성 
        
    def __get__(self, instance, owner):
        print("get")
        return self.value                 # 디스크립터 객체의 값을 조회
    
    def __set__(self, instance, value):
        print("set")
        self.value = value                # 디스크립터 객체의 값을 갱신
        
    def __delete__(self, instance):
        print("delete")
        del self.value

## 1-2 사용자 정의 클래스에 디스크립트 사용하기 


###  디스크립트 객체는 반드시 클래스 속성에 할당 

- 디스크립터 객체는 반드시 클래스 속성으로 정의한다. 


In [3]:
class A:
    value = Descriptor(10) # instance
    value1 = Descriptor(10) # instance
    
    def __init__(self, value=10, value1=10) :
        self.value = value                       # 객체 속성이 생기지 않고 디스크립터 객체를 먼저 접근
        self.value1 = value1                     # 그래서 set 까지 지정된 디스크립터는 항상 먼저 접근이 된다 

## 1-3  객체를 생성

- 객체를 생성할 때 디스크립터 객체를 가진 속성의 값을 갱신한다.
- 처음 지정한 값과 갱신하는 값이 동일하다.  


In [4]:
a = A()

set
set


### 실제 객체에는 아무 속성이 없다. 

In [5]:
a.__dict__

{}

## 1-4 디스크립트 객체에 있는  속성 조회하기

### 값을 조회하기

- 디스크립터는 클래스 내부의 속성을 확인하고 이를 이용해서 내부의 메소드를 호출해서 처리
- 클래스 A의 네임스페이스를 조회하면 두 개의 디스크립터 속성이 만들어져 있다.

In [6]:
A.__dict__

mappingproxy({'__module__': '__main__',
              'value': <__main__.Descriptor at 0x103ed4290>,
              'value1': <__main__.Descriptor at 0x103ed6c50>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

### 클래스 내의 두 개의 디스크립스 속성에 접근


In [8]:
type(a).__dict__['value']

### 속성은 디스크립트 객체이므로  스페셜 메소드 확인 

In [9]:
type(a).__dict__['value'].__get__

<bound method Descriptor.__get__ of <__main__.Descriptor object at 0x103ed4290>>

### 함수처럼 바인드해서 읽는다. 

In [10]:
type(a).__dict__['value'].__get__(a, type(a))

get


10

### 실제 객체의 속성을 조회하는 것도 내부적으로는 위의 방식으로 처리 

In [11]:
a.value     # 디스크립터 클래스의 __get__ 메소드를 조회한다. 

get


10

## 1-5  디스크립터는 클래스의 속성 갱신

- 값을 디스크립터 객체에 넣고 처리해서 동일하게 공유되는 것을 볼 수 있다. 
- 

### 또 하나의 객체를 읽는다 

In [12]:
b = A()

In [13]:
b.value

get


10

### 다른 객체의 디스크립트 속성을 갱신하면 두 개의 객체의 값이 같아짐

-  디스크립트 객체의 속성에 값을 보관하면 모든 객체가 공유처리

In [14]:
type(a).__dict__['value'].__set__(a, 12)

set


In [15]:
a.value, b.value

get
get


(12, 12)

In [16]:
a.value1 = 13  

set


In [17]:
a.value1, b.value1

get
get


(13, 13)

## 1-5   디스크리터 속성 삭제 

### 속성 삭제 

In [5]:
type(a).__dict__['value'].__delete__(a)

delete


###  클래스의 속성 확인 

In [7]:
type(a).__dict__

mappingproxy({'__module__': '__main__',
              'value': <__main__.Descriptor at 0x107837d90>,
              'value1': <__main__.Descriptor at 0x107837950>,
              '__init__': <function __main__.A.__init__(self, value=10, value1=10)>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

### 실제  속성을 접근하면 디스크립터 객체의 속성 삭제 

In [8]:
try : 
    a.value
except Exception as e :
    print(e)

get
'Descriptor' object has no attribute 'value'


### 다시 삭제하면 디스크립터 객체의 속성이 삭제

In [9]:
del a.value1 

delete


In [10]:
type(a).__dict__

mappingproxy({'__module__': '__main__',
              'value': <__main__.Descriptor at 0x107837d90>,
              'value1': <__main__.Descriptor at 0x107837950>,
              '__init__': <function __main__.A.__init__(self, value=10, value1=10)>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [11]:
try : 
    a.value1
except Exception as e :
    print(e)

get
'Descriptor' object has no attribute 'value'


## 1-6 다시 세팅하면 디스크립터가 작동되어 처리됨 

- 클래스 내의 속성은 있어서 다시 세팅하면 디스크립터 속성이 만들어진다

In [12]:
a.value =123

set


In [13]:
a.value

get


123

In [14]:
type(a).__dict__['value1'].__set__(a,999)

set


In [15]:
a.value1

get


999

## 1-7.  디스크립터 프로토콜이 없다면..

- 속성을 관리하는 클래스를 정의하고 메인 클래스의 속성에 객체를 생성한다.
- 메인 클래스 속성 즉 객체에 속성을 추가한다.
- 이 속성에 정보를 조회한다.
- 점연산자를 연속적으로 사용해야 한다.
- 디스크립터 클래스를 만들고 처리하는 것보다 더 복잡한 구조를 가진다.

### 일반 클래스를 정의한다. 

- `__getattribute__` 는 속성을 접근하는 연산자 즉 점(.) 연산자 

In [20]:
class D:
    def __getattribute__(self, name):            # 객체의 속성을 접근하는 연산를 구현
        print("getattribute")
        return super().__getattribute__(name)


### 클래스 내부의 속성에 객체를 할당한다.

-  클래스 속성에 일반 클래스의 객체를 할당한다

In [21]:
class A:
    value = D()

### 객체 내부의 value 속성은 객체이미므로 이 객체의 속성 data를 추가한다.

- 객체를 생성하고 클래스 속성(다른 클래스의 객체)에 특성 속성을 추가한다. 

In [22]:
a = A()
a.value.data = 1

### 객체 내부의 data 값을 조회한다. 

-  계속 점연산으로 속성 값을 확인할 수 있다. 

In [23]:
a.value.data

getattribute


1

## 1-8.   읽기 전용  디스크립터 


### get 프로토콜만 정의하기 

`__get__`  만 가진 디스크립터

- 메인 클래스의 속성을 읽기 전용으로만 만들 수 있다.

In [29]:
class FirstDescriptor:
    def __init__(self, value):
        self.value = value
        
    def __get__(self, instance, owner):
        print("get")
        return self.value
    

class A:
    a = FirstDescriptor(3)

In [30]:
a = A()
a.a

get


3

### set 프로토콜을 에러 처리 하기 

- `__set__` 을 정의했지만 예외를 발생시켜서 읽기 전용으로 만들 수 있다.  



In [31]:
class Descriptor:
    def __init__(self, value):
        self.value = value
        
    def __get__(self, instance, onwer):
        print("get")
        return self.value
    
#     def __set__(self, instance, value):
#         print("set")
#         self.value = value  # 쓰기 가능
        
    def __set__(self, instance, value):
        # 읽기전용
        print("set")
        raise AttributeError("read only")
        


In [32]:
class A:
    a = Descriptor(3)

In [33]:
a = A()

In [34]:
a.a

get


3

In [35]:
try : 
    a.a = 12
except Exception as e :
    print(e)

set
read only


# 2. 객체 별로 별도의 속성을 관리하는 디스크립터 만들기

### 디스크립터 클래스는 실제 클래스의 객체를 접근하는 역할로 한정한다.

### 디스크립터 클래스를 정의할 때 전달받는 객체의 속성에 값을 저장한다. 



## 2-1 실제 클래스의 객체에 값을 보관

### 디스크립터에 다른 클래스의 객체 정보를 활용  

In [33]:
class descriptor:
    def __get__(self, instance, owner):
        print('get')
        return instance.value
    
    def __set__(self, instance, value):
        print('set')
        instance.value = value


### 클래스에 디스크립터 속성을 정의한다

In [34]:
class A:
    x = descriptor()

### 객첼ㄹ 생성한다.

In [35]:
a11 = A()

In [36]:
a11.__dict__

{}

### 속성을 추가한다.

In [37]:
a11.x = 123

set


In [38]:
a11.__dict__

{'value': 123}

In [39]:
try :
    a11.x
except Exception as e :
    print(e)

get


## 2-2 인스턴스 객체 내에 속성 저장하고 처리하기

### 디스크립터 객체 생성

- 객체 생성자에 인스턴스 객체에 저장될 이름을 지정
- 조회와 갱신을 인스턴스 객체의 속성을 접근해서 처리하도록 수정 

In [41]:
class descriptor1:
    
    def __init__(self, name):
        self.name = "_" + name 
        
    def __get__(self, instance, owner):
        print('get', instance)
        return instance.__dict__.get(self.name,0) 
    
    def __set__(self, instance, value):
        print('set')
        instance.__dict__[self.name] = value



### 실제 사용 클래스 정의 

In [42]:
class AA:
    x = descriptor1('x')

### 객체 생성하기 

In [43]:
aa = AA()

### 객체의 속성 조회하기

- 현재 속성이 없지만 기본값으로 세팅

In [44]:
aa.x

get <__main__.AA object at 0x107dd2490>


0

### 속성의 값을 처음으로 입력

In [45]:
aa.x = 100

set


In [46]:
aa.__dict__

{'_x': 100}

In [47]:
aa.x

get <__main__.AA object at 0x107dd2490>


100

## 3.   메인 클래스 속성에서 이름 가져오기

### 클래스 디스크립터 정의할 때 클래스 속성이름 가져오기

-  `__set_name__` 스페셜 메서드를 사용해서 사용 클래스의 속성 이름을 전달 받는다

In [50]:
class descriptor2:
    
    def __set_name__(self, owner, name ):
        self.name = "_"+ name
        
    def __get__(self, instance, owner):
        print('get', instance)
        return instance.__dict__.get(self.name,0) 
    
    def __set__(self, instance, value):
        print('set')
        instance.__dict__[self.name] = value

###  사용 수퍼 클래스에 디스크립터 속성을 2개 정의 

In [51]:
class AAA:
    x = descriptor2()
    y = descriptor2()
    
    def __init__(self, x,y) :
        self.x = x
        self.y = y

In [52]:
aaa = AAA(10,20)

set
set


In [53]:
aaa.x = 100

set


In [54]:
aaa.x

get <__main__.AAA object at 0x107d61150>


100

## 4.  상속해서 디스크립터 사용하기 

### 실제 자식 클래스에 디스크립터 속성 1개 정의
- 수퍼 클래스의 생성자를 호출해서 2개의 값을 세팅 

In [55]:
class BBB(AAA) :
    z = descriptor2()
    def __init__(self,x,y,z) :
        super().__init__(x,y)
        self.z = z

### 클래스 정보를 확인 

In [56]:
BBB.__dict__

mappingproxy({'__module__': '__main__',
              'z': <__main__.descriptor2 at 0x107dbb4d0>,
              '__init__': <function __main__.BBB.__init__(self, x, y, z)>,
              '__doc__': None})

### 객체를 생성하기

In [58]:
bbb = BBB(10,20,30)

set
set
set


### 속성을 확인하기 

In [59]:
bbb.x, bbb.y, bbb.z

get <__main__.BBB object at 0x10448bd50>
get <__main__.BBB object at 0x10448bd50>
get <__main__.BBB object at 0x10448bd50>


(10, 20, 30)

## 5. 조회할 때 속성을 초기화하기

### 디스크립터 클래스 정의할 때 조회 내에 갱신메서드를 사용해서 속성 초기화 지정

In [60]:
class descriptor3:
    
    def __set_name__(self, owner, name ):
        self.name = "_"+ name
        
    def __get__(self, instance, owner):
        print('get', instance)
        return instance.__dict__.setdefault(self.name,0) 
    
    def __set__(self, instance, value):
        print('set')
        instance.__dict__[self.name] = value

### 사용 클래스를 정의

In [61]:
class AAAA :
    x = descriptor3()

### 객체를 생성하기

In [62]:
aaaa = AAAA()

### 속성을 조회하기

In [63]:
aaaa.x

get <__main__.AAAA object at 0x1044b6310>


0

### 객체의 속성이 초기화처리 

In [64]:
aaaa.__dict__

{'_x': 0}

### 속성값 변경 하기

In [65]:
aaaa.x =100

set


In [66]:
aaaa.__dict__

{'_x': 100}

In [67]:
aaaa.x

get <__main__.AAAA object at 0x1044b6310>


100

## 5.  함수를 저장해서 처리하는 디스크립터 만들기 

- 디스크립터 클래스를 만들고 데코레이터를 처리할 수 있다.
- 이름으로 접근하면 내부으 함수가 실행된다.


### 디스크립터 조회할 때 저장된 함수를 반환한다

In [68]:
class descriptor:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        print("get")
        return self.func(instance) # 메소드는 첫번째 인자로 self 자기자신을 받아야되기 때문에

### 디스크립터로 메소드를 데코레이터 처리한다.

In [69]:
class A:
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c
    
    @descriptor
    def func(self):
        return self.a + self.b + self.c


In [70]:
a = A(1, 2, 3)

In [71]:
a.func # __call__ X

get


6

### 함수의 인자를 받도록 수정

- 부분 함수 처리를 위해 pattial 로 처리한다. 
- 이름으로 조회하면 부분함수가 반환되고 메소드의 인자를 추가적으로 넣어서 처리할 수 있다

In [72]:
from functools import partial


class descriptor:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        print('get')
        return partial(self.func, instance)

In [73]:
class A:
    @descriptor
    def func(self, a, b, c):
        return a + b + c

In [74]:
a = A()

In [75]:
a.func(1, 2, 3)

get


6

## 6. 도전 : 디스크립터로 구현

### 인스턴스를 생성할때 반지름을 입력받는 원 객체 / 원의 속성을 가지고 있는..

> 반지름은 mutable    
> 그 외에 지름, 둘레, 넓이는 immutable

### 함수를 저장하는 방식으로

### 갱신가능한 디스크립터 정의

In [76]:
class MutableAttribute:
    def __init__(self, value=None):
        self.value = value
        
    def __get__(self, instance, owner):
        return self.value
    
    def __set__(self, instance, value):
        self.value = value
        
    def __delete__(self, instance):
        del self.value



### 갱신 불가능한 디스크립터 정의

In [77]:
class ImmutableAttribute:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        return self.func(instance)
    
    def __set__(self, instance, value):
        raise AttributeError("read only")
        
    def __delete__(self, instance):
        raise AttributeError("read only")

### 메인 클래스 정의하고 메소드를 변경불가능하도록 처리

In [78]:
class Circle:
    pi = 3.1415
    radius = MutableAttribute(10)
    diameter = ImmutableAttribute(lambda self : self.radius * 2)
    
#     @ImmutableAttribute
#     def diameter(self):
#         return self.radius * 2
    
    @ImmutableAttribute
    def circumference(self):
        return self.radius * self.pi * 2
    
    @ImmutableAttribute
    def area(self):
        return self.radius **2 * 2

In [79]:
c = Circle()

In [80]:
c.radius

10

In [81]:
c.diameter

20

In [82]:
c.area

200

In [83]:
c.radius = 100

In [84]:
c.area

20000

In [85]:
try :
    c.area = 123
except Exception as e :
    print(e)

read only
